In [1]:
import pandas as pd
import datetime as datetime

In [2]:
bookings = pd.read_csv('booking_data.csv').drop(['Unnamed: 0'], axis = 1)
signup_data = pd.read_csv('signup_data.csv').drop(['Unnamed: 0'], axis = 1)

In [3]:
recent_date = '03/08/2022'
days = [14,30,60,90]
null_value = 0

##### Required Columns
- user_id
- user_type
- Signup date
- first ride date
- last ride date
- total rides so far
- total rides in last 90 days
- total rides in first 30 days
- total rides in second 30 days
- total rides in third 30 days


In [6]:
df = bookings.merge(signup_data, left_on='user_id', right_on='user_id')

In [8]:
completed_bookings = df.loc[df.status == 'completed'].copy()
rides_df = completed_bookings[['user_id', 'booking_created_on']].groupby('user_id').agg(['min', 'max', 'count'])
columns = ['first_ride_date', 'last_ride_date', 'Total_rides']
rides_df.columns = columns
result = pd.merge(signup_data,rides_df, how = 'left', on="user_id")
for col in columns:
    result[col] = result[col].fillna(null_value)

In [9]:
result.head()

,user_id,signup_time_stamp,first_ride_date,last_ride_date,Total_rides
0,3574034,2020-01-23 21:28:09,2021-09-01 00:07:07.344,2021-09-29 20:50:13.883,9.0
1,1086382,2019-06-07 21:58:37,2021-09-01 00:07:17.483,2021-09-02 21:02:59.180,3.0
2,859987,2019-04-21 12:08:48,2021-09-01 00:08:09.544,2022-02-11 00:13:12.439,9.0
3,4896659,2021-01-12 14:01:22,2021-09-01 00:08:37.778,2022-03-09 11:30:03.527,75.0
4,5239248,2021-08-22 22:32:44,2021-09-01 10:26:10.552,2021-09-30 17:02:01.858,50.0


In [10]:

#getting ride details for all the days
completed_bookings["booking_created_on"] =  pd.to_datetime(completed_bookings["booking_created_on"], format='%Y-%m-%d %H:%M:%S.%f')
for x in days:
    temp_df = completed_bookings[completed_bookings["booking_created_on"] >= (pd.to_datetime(recent_date) - pd.Timedelta(days=x))]
    temp_df = temp_df.groupby('user_id')['status'].size().reset_index(name=f'ride_in_last_{x}_days')
    result = pd.merge(result ,temp_df, how = 'left', on="user_id")
    result[f'ride_in_last_{x}_days'] = result[f'ride_in_last_{x}_days'].fillna(null_value)


In [11]:
result.head()

,user_id,signup_time_stamp,first_ride_date,last_ride_date,Total_rides,ride_in_last_14_days,ride_in_last_30_days,ride_in_last_60_days,ride_in_last_90_days
0,3574034,2020-01-23 21:28:09,2021-09-01 00:07:07.344,2021-09-29 20:50:13.883,9.0,0.0,0.0,0.0,0.0
1,1086382,2019-06-07 21:58:37,2021-09-01 00:07:17.483,2021-09-02 21:02:59.180,3.0,0.0,0.0,0.0,0.0
2,859987,2019-04-21 12:08:48,2021-09-01 00:08:09.544,2022-02-11 00:13:12.439,9.0,0.0,1.0,2.0,2.0
3,4896659,2021-01-12 14:01:22,2021-09-01 00:08:37.778,2022-03-09 11:30:03.527,75.0,12.0,25.0,40.0,55.0
4,5239248,2021-08-22 22:32:44,2021-09-01 10:26:10.552,2021-09-30 17:02:01.858,50.0,0.0,0.0,0.0,0.0


In [12]:
def diff(a,b):
    if  b != 0:
        return a-b
    else:
        return a 

In [13]:
result['rides_in_second_30_days'] = result.apply(lambda x:  diff(x['ride_in_last_60_days'], x['ride_in_last_30_days']), axis=1)
result['rides_in_third_30_days'] = result.apply(lambda x:  diff(x['ride_in_last_90_days'], x['ride_in_last_60_days']), axis=1)
result['user_type'] = 'NA'

In [14]:
# no ride till now
result.loc[result.first_ride_date == 0, 'user_type'] = 'Never took a ride'

In [15]:
#took a ride before 90 days and never took a ride since then
result.loc[(result.ride_in_last_90_days == 0)
            & (result.Total_rides != 0), 'user_type'] = 'Churned'

In [16]:
#took his first ride in last 30 days
result.loc[(result.ride_in_last_30_days == result.Total_rides)
            &(result.Total_rides != 0),'user_type' ] = 'New Rider'

In [17]:
# took and a ride in either 2nd or 3rd 30 days
result.loc[((result.rides_in_second_30_days != 0)
            | (result.rides_in_third_30_days != 0))
            & (result.ride_in_last_30_days == 0), 'user_type'] = 'Dormant'

In [18]:
# took less than 30 rides in last 90 days but not in last 30 days
result.loc[(result.ride_in_last_90_days <= 30)
            & (result.ride_in_last_30_days != 0), 'user_type'] = 'Irregular'

In [19]:
#took a ride in last 14 days, completed 30-60 rides in 90 days and >10 rides in last 3 months
result.loc[(result.ride_in_last_14_days > 0)
            & (result.ride_in_last_90_days > 30)
            & (result.ride_in_last_90_days <= 60)
            & (result.ride_in_last_30_days >=10)
            & (result.rides_in_second_30_days >=10)
            & (result.rides_in_third_30_days >=10), 'user_type'] = 'Regular'

In [20]:
#took a ride in last 14 days, completed >60 rides in 90 days and >20 rides in last 3 months
result.loc[(result.ride_in_last_14_days > 0)
            & (result.ride_in_last_90_days > 60)
            & (result.ride_in_last_30_days >=20)
            & (result.rides_in_second_30_days >=20)
            & (result.rides_in_third_30_days >=20), 'user_type'] = 'Power'

In [21]:
result.user_type.value_counts()

Irregular            75002
Churned              70218
Dormant              55870
never_took_a_ride    13640
NA                    4250
Power                  946
Regular                551
New Rider              366
Name: user_type, dtype: int64